# Single channel vs classic gateways

In [ ]:
# SET your current working directory!
cwd = "/home/jovyan/work/pdr_duplication"
overwrite = False

In [ ]:
import os
import sem
import numpy as np
import scipy as sp
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots

plt.style.use(["science", "ieee"])
plt.rcParams.update({"figure.dpi": "200"})

# Create our SEM campaign
ns_3_dir = os.environ["NS3DIR"]
script = "clues"
results_dir = cwd + "/results"
plots_dir = cwd + "/plots"

os.makedirs(plots_dir, exist_ok=True)
campaign = sem.CampaignManager.new(ns_3_dir, script, results_dir, overwrite=overwrite)
overwrite = False
runs = 30

# Define the global parameter space we are interested in exploring
globals = {
    "hoursNum": 10,
    "sideLength": 9200,
    "devNum": [
        100,
        1000,
        2000,
        3000,
        4000,
        5000,
        6000,
        7000,
        8000,
        9000,
        10000,
    ],
}

In [ ]:
# If the previous cell fails due to changes in the simulation code,
# you can re-run the previous cell after running this one to fix it.
##################### WARNING! #####################
# It will overwrite all existing simulation results,
# so back them up beforehand if you need them.
overwrite = True

Utility functions

In [ ]:
def plot_size(h, gold=False):
    letter = 1.294
    r = sp.constants.golden if gold else letter
    w = h * r
    return (w, h)

## Run experiments

In [ ]:
# Define the local parameter space we are interested in exploring
params = {
    "gwNum": [
        10,
        100,
        1000,
    ],
    "scenario": ["SF7Single", "FullRand"],
}
params.update(globals)

# Run simulations with the above parameter space
print(f"• Running missing simulations for param space: {params}.")
campaign.run_missing_simulations(params, runs)

In [ ]:
# Define the local parameter space we are interested in exploring
params = {
    "gwNum": 10,
    "scenario": "Classic",
}
params.update(globals)

# Run simulations with the above parameter space
print(f"• Running missing simulations for param space: {params}.")
campaign.run_missing_simulations(params, runs)

## Packet delivery outcomes

In [ ]:
@sem.utils.yields_multiple_results
@sem.utils.output_labels(["Sent", "_field", "_value"])
def get_outcomes(result):
    """
    Extract the packet delivery outcomes from outcomes.csv
    """
    df = pd.read_csv(
        StringIO(result["output"]["outcomes.csv"]), header=None
    ).T.set_axis(["f", "v"], axis=1)
    df.insert(0, "s", df.iloc[0, 1])
    return df.drop(0).astype({"s": "int64", "v": "int64"}).to_numpy().tolist()


filename = "outcomes"
print(f"• Collecting {filename} results...")
r = campaign.get_results_as_dataframe(
    get_outcomes, params=globals, verbose=True, parallel_parsing=True
)
r["percent"] = r["_value"].to_numpy() / r["Sent"].to_numpy() * 100
r.to_csv(cwd + f"/{filename}.csv")
r

### Packet Delivery Ratio: share of packets that have been correctly received

In [ ]:
filename = "pdr"
print(f"• Generating {filename} plot...")
# Load and filter data
df = pd.read_csv(cwd + "/outcomes.csv")
df = df[df["_field"] == "Received"]
for o, n in zip(["SF7Single", "FullRand", "Classic"], ["SCSF7", "Random", "LoRaWAN"]):
    df.loc[df["scenario"] == o, "scenario"] = n
## Plot data
w, h = plot_size(3)
fig, ax = plt.subplots(figsize=(w, h + 0.75))
sns.lineplot(
    data=df,
    y="percent",
    x="devNum",
    hue="scenario",
    style="gwNum",
    err_style="bars",
    hue_order=["SCSF7", "Random", "LoRaWAN"],
    palette=sns.cubehelix_palette(3, reverse=True),
)
## Global settings
ax.set_ylim(ymin=0)
ax.set_xlabel("Number of devices")
ax.set_ylabel(r"Packet delivery ratio [\%]")
# Legend
labels = [i + ", " + j for i in ["SCSF7", "Random"] for j in ["10", "100", "1000"]] + [
    "LoRaWAN, 10"
]
leg = plt.legend(
    labels, title="", loc="upper center", bbox_to_anchor=(0.445, -0.15), ncols=3
)
vboxes = leg._legend_box._children[1]._children
vboxes[1]._children.append(vboxes[2]._children.pop(0))
# Save to file
plt.tight_layout()
plt.savefig("{0}/{1}.{2}".format(plots_dir, filename, "pdf"))
plt.savefig("{0}/{1}.{2}".format(plots_dir, filename, "png"))

## Packet duplication statistics 

In [ ]:
@sem.utils.output_labels(["Duplications"])
def get_dup_stats(result):
    """
    Extract duplication statistics from duplication.csv
    """
    return (
        pd.read_csv(StringIO(result["output"]["duplication.csv"]))
        .to_numpy()
        .tolist()[0]
    )


filename = "duplication"
print(f"• Collecting {filename} results...")
r = campaign.get_results_as_dataframe(
    get_dup_stats, params=globals, verbose=True, parallel_parsing=True
)
r.to_csv(cwd + f"/{filename}.csv")
r

### Packet duplication index: total number of receptions divided by the number of unique packets sent

In [ ]:
filename = "duplication"
print(f"• Generating {filename} plot...")
# Load and filter data
df = pd.read_csv(cwd + "/duplication.csv")
for o, n in zip(["SF7Single", "FullRand", "Classic"], ["SCSF7", "Random", "LoRaWAN"]):
    df.loc[df["scenario"] == o, "scenario"] = n
## Plot data
w, h = plot_size(3)
fig, ax = plt.subplots(figsize=(w, h + 0.75))
sns.lineplot(
    data=df,
    y="Duplications",
    x="devNum",
    hue="scenario",
    style="gwNum",
    err_style="bars",
    hue_order=["SCSF7", "Random", "LoRaWAN"],
    palette=sns.cubehelix_palette(3, reverse=True),
)
# Global settings
ax.set_xlabel("Number of devices")
ax.set_ylabel("Avg. packet duplications")
ax.set_yscale("log")
labels = [i + ", " + j for i in ["SCSF7", "Random"] for j in ["10", "100", "1000"]] + [
    "LoRaWAN, 10"
]
leg = plt.legend(
    labels, title="", loc="upper center", bbox_to_anchor=(0.445, -0.15), ncols=3
)
vboxes = leg._legend_box._children[1]._children
vboxes[1]._children.append(vboxes[2]._children.pop(0))
# Export
plt.tight_layout()
plt.savefig("{0}/{1}.{2}".format(plots_dir, filename, "pdf"))
plt.savefig("{0}/{1}.{2}".format(plots_dir, filename, "png"))